## PyNWB reading files
<br> Here are some tutorials I found useful during our data packaging:
<br> https://pynwb.readthedocs.io/en/stable/tutorials/domain/ecephys.html#sphx-glr-tutorials-domain-ecephys-py
<br> https://pynwb.readthedocs.io/en/stable/tutorials/general/plot_read_basics.html#

In [1]:
import os
import sys
import time
from datetime import datetime
from dateutil.tz import tzlocal

import numpy as np
from pynwb import NWBFile, NWBHDF5IO

## Specify file location

In [2]:
# filepath = r"T:\zap-n-zip\eLife_manuscript_NWBfiles\000458\sub-569069\sub-569069_ses-20210312_behavior+ecephys.nwb"
filepath = r"E:\data_files\2021-02-09_546513_1.nwb"

## Open the NWB file in read mode

In [3]:
io = NWBHDF5IO(filepath, mode="r")
nwbfile = io.read()
nwbfile

root pynwb.file.NWBFile at 0x1257161546504
Fields:
  devices: {
    Neuropixels Probe <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    ProbeA Electrode Group <class 'pynwb.ecephys.ElectrodeGroup'>,
    ProbeC Electrode Group <class 'pynwb.ecephys.ElectrodeGroup'>,
    ProbeE Electrode Group <class 'pynwb.ecephys.ElectrodeGroup'>,
    ProbeF Electrode Group <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: Test description of the mouse
  experimenter: ['Andrew Shelton']
  file_create_date: [datetime.datetime(2024, 7, 1, 17, 33, 11, 358872, tzinfo=tzoffset(None, -25200))]
  identifier: 2021-02-09_546513_1
  institution: Allen Institute
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  lab: Neural Dynamics
  processing: {
    10ms_pulse_psth <class 'pynwb.base.ProcessingModule'>,
    1s_cosine_psth <class 'pynwb.base.ProcessingModule'>,
    spike_times <class 'pynwb.base.Proce

See above the nwbfile print out shows "units" at the very bottom. That is where the spikes are stored.

### Accessing units and spikes

In [4]:
all_units_df = nwbfile.units.to_dataframe()
all_units_df.head()

,unitID,sessionID,mouseID,genotype,recording,recording_date,probe,optotagged,ct,amp,dur,PTR,prePTR,repol_slope,recov_slope,avg_ISI,baseline_FR,footprint
id,,,,,,,,,,,,,,,,,,
0,exp0_cell1,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,C,0,RS,91.206184,0.866667,0.383150,0.000652,194.514775,-16.795936,146.906211,7.5125,31
1,exp0_cell2,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,C,0,RS,198.866773,0.733333,0.259419,-0.000652,741.979144,-8.722476,765.419539,1.3375,18
2,exp0_cell4,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,C,0,RS,218.769131,0.700000,0.492346,0.004151,569.239850,-63.944546,1128.509923,0.8375,31
3,exp0_cell11,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,C,0,RS,148.146073,0.666667,0.687285,0.477277,273.145229,-45.870562,685.405250,1.3250,44
4,exp0_cell12,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,C,0,FSl,150.562230,0.266667,0.664475,-0.000464,589.126340,-74.325952,64.932023,12.8750,26


In [5]:
len(all_units_df)

64

In [7]:
all_units_df.tail()

,unitID,sessionID,mouseID,genotype,recording,recording_date,probe,optotagged,ct,amp,dur,PTR,prePTR,repol_slope,recov_slope,avg_ISI,baseline_FR,footprint
id,,,,,,,,,,,,,,,,,,
59,exp1_cell63,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,E,0,RS,125.977079,0.600000,0.519463,0.000190,358.886759,-25.924361,332.291181,2.9750,24
60,exp1_cell70,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,E,0,RS,120.550562,0.700000,0.442750,0.045640,289.615886,-24.720606,1543.057444,0.6875,36
61,exp1_cell74,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,E,0,RS,416.667448,0.666667,0.309317,0.002471,1535.887666,-67.557998,1793.512426,0.5375,68
62,exp1_cell83,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,E,0,RS,118.210275,0.966667,0.263264,0.038669,307.809805,-13.637278,972.840405,0.9000,47
63,exp1_cell84,2021-02-09_546513,546513,VIP-Cre;Ai32,1,2021-02-09,E,0,RS,109.955767,0.833333,0.329936,0.038237,243.510253,-18.495995,2152.318569,0.3875,39


In [9]:
probes = np.unique(all_units_df['probe'].values)
print(probes)

['C' 'E']


### Check trials

In [10]:
trial_df = nwbfile.trials.to_dataframe()
trial_df.head()

,start_time,stop_time,trial_levels,trial_conditions
id,,,,
0,40.95940,41.95940,1.4,3.0
1,43.11986,44.11986,1.4,3.0
2,45.02147,46.02147,1.4,3.0
3,46.73219,47.73219,1.4,2.0
4,48.91272,49.91272,1.4,2.0


### Check what's in the processed section

In [12]:
print(nwbfile.processing.keys())

dict_keys(['10ms_pulse_psth', '1s_cosine_psth', 'spike_times', 'waveforms'])


In [14]:
print(nwbfile.processing['spike_times'])

spike_times pynwb.base.ProcessingModule at 0x1257161223240
Fields:
  data_interfaces: {
    spike_times_unit0 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit1 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit10 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit11 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit12 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit13 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit14 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit15 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit16 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit17 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit18 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit19 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit2 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit20 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit21 <class 'pynwb.base.TimeSeries'>,
    spike_times_unit22 <class 'pynwb.base.TimeSeries'>,
    spike_times_uni

### Close any files

In [15]:
io.close()